# Exam number: Y3892609

Install packages

In [71]:
%pip install torch pandas matplotlib scikit-learn numpy deap seaborn scipy -q

# Data examination and pre-processing

In [4]:
import numpy as np
import pandas as pd
from IPython.display import display, Latex

data = pd.read_csv("eScooterDemand.csv")

## Addressing the data observations

### Convert "Yes/No" cells to numeric values

In [5]:
data["Public Holiday"] = data["Public Holiday"].apply(lambda x: 1 if x == "Yes" else 0)
data["HireAvailable"] = data["HireAvailable"].apply(lambda x: 1 if x == "Yes" else 0)
display(Latex(data[["Public Holiday", "HireAvailable"]].head().to_latex()))

### One-hot encode season data

In [6]:
for season in ["Winter", "Autumn", "Spring", "Summer"]:
    data[season] = data["Season"].apply(lambda x: 1 if x == season else 0)
del data["Season"]
display(Latex(data[["Winter", "Autumn", "Spring", "Summer"]].head().to_latex()))

### Break date column into day, month and year columns

In [7]:
data["day"] = data["Date"].apply(lambda x: int(x.split("/")[0]))
data["month"] = data["Date"].apply(lambda x: int(x.split("/")[1]))
data["year"] = data["Date"].apply(lambda x: int(x.split("/")[2]))
del data["Date"]
display(Latex(data[["day", "month", "year"]].head().to_latex()))

### Scale numeric columns
This will keep the range of the data manageable, reducing the need for the algorithm to "balance" data of different scales.

In [8]:
from sklearn.preprocessing import minmax_scale

numeric_columns = [
    "Temp",
    "Humidity",
    "Wind speed",
    "Hour",
    "Visibility",
    "Dew point",
    "Sunshine",
    "Rain",
    "Snow",
    "day",
    "month",
    "year",
]
data[numeric_columns] = minmax_scale(data[numeric_columns])
display(
    Latex(
        data[list(set(numeric_columns) - {"day", "month", "year"})]
        .head()
        .to_latex(float_format="%.2f")
    )
)

### Remove rows where the hire scheme is not running
This will also allow for deleting the `HireAvailable` column since it will now be guaranteed to be available

In [9]:
data_when_available = data.drop(data[data["HireAvailable"] == 0].index)
del data_when_available["HireAvailable"]
num_dropped_rows = len(data) - len(data_when_available)
print(
    f"Removed {num_dropped_rows} rows ({round((num_dropped_rows / len(data)) * 100, 2)}% of all rows)"
)

The numbers above suggest that the number of readings taken when the scheme was not operating did not represent a significant proportion of all readings and is therefore unlikely to affect the model's performance on the rest of the data.

### Final observations
Now that the data is scaled, the range of values of the `Count` column is significantly greater, requiring large weights in individuals.
While not necessarily an issue, converting the data to a lower range may help the individuals keep their representation to a smaller range of values, reducing the search space.
A potential way of reducing the range is applying a natural log to the `Count` column. Given that the maximum in this dataset is 3556, the maximum log value will be around 8, significantly reducing the available range. 
A possible issue may arise with counts of 0 and 1 - the log of 1 is 0 and log of 0 is undefined. A possible solution is to simply not convert counts of zero. While calculating the original value from the converted column could lead to an error of 1, the scale of the original `Count` column makes it possible to argue that an error of one is not significant in the scale of the average number of daily rentals.

For additional confidence in the above, count the number of times when `Count` was either 0 or 1

In [10]:
count_equal_to_zero = len(data_when_available[data_when_available["Count"] == 0])
count_equal_to_one = len(data_when_available[data_when_available["Count"] == 1])
print(
    f"Count = 0 on {count_equal_to_zero} observations\nCount = 1 on {count_equal_to_one} observations"
)

In [11]:
data_when_available["Count"] = data_when_available["Count"].apply(
    lambda x: 0 if x == 0 else np.log(x)
)
display(Latex(data_when_available["Count"].head().to_latex(float_format="%.5f")))

In [12]:
print(
    f"New range for the Count column - min: {data_when_available['Count'].min()}, max: {data_when_available['Count'].max()}"
)

## Investigating the degree to which other columns affect the count

In [13]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(35, 16))
ax.bar_label(
    ax.bar(
        list(map(str.capitalize, data_when_available.drop("Count", axis=1).columns)),
        data_when_available.corr("pearson")["Count"]
        .drop("Count")
        .abs()
        .sort_values(ascending=False),
    ),
    fontsize=18,
)
_ = ax.set_title(
    "Absolute pearson correlation between columns and scooter count", fontsize=24
)
ax.tick_params(axis="both", which="major", labelsize=18)

It is clear that the hour and air temperature have the strongest correlation with the number of rentals.
This plot suggests possible design choices for individuals in the next section. While the size of the individuals in a neural network representation may not differ significantly with less "important" columns (i.e. with more input neurons), a tree-based genetic programming approach can benefit significantly from a reduced number of branches. In that case, a technique like PCA could be used to extract the most informative features from the data while keeping the trees as narrow as possible.

Overall, the large number of available variables suggest evolving a neural network as a strong choice for this task, as the large amounts of internal connections will allow the individuals to model relationships between the variables that would be hard to capture otherwise - in a genetic programming approach, the individuals are likely to have extremely large heights, leading to computationally expensive evaluation functions and slowing down the training loop. Furthermore, by comparing the evolved network to one trained using a conventional backpropagation algorithm, the success of the evolutionary algorithm can be easily evaluated and compared to other methods.

# Neural network approach - investigating network architectures

It may be beneficial to look at the performance of a neural network trained the conventional way to get an idea of the possible performance of one trained using a genetic algorithm

Split data into a "training" and testing set - the training set will be used to evaluate the fitness of the individuals during the evolution process and the testing portion will be used to evaluate the best individual's performance on unseen data

In [14]:
from sklearn.model_selection import train_test_split

X, y = data_when_available.drop("Count", axis=1), data_when_available["Count"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.8,
    random_state=222,  # for repeatable results
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

One hidden layer

In [15]:
import torch

X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = (
    torch.from_numpy(np.float32(X_train)),
    torch.from_numpy(np.float32(X_test)),
    torch.from_numpy(np.float32(y_train)).unsqueeze(1),
    torch.from_numpy(np.float32(y_test)).unsqueeze(1),
)

model_1_h_l = torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], 256, bias=False),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 1, bias=False),
    torch.nn.ReLU(),  # The number of rentals can never go below zero, prevent the model from outputting negative numbers
)

optim = torch.optim.Adam(model_1_h_l.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
num_epochs = 4000

for epoch in range(num_epochs + 1):
    # Pass training data through model
    y_predict = model_1_h_l(X_train_tensor)
    # Compute BCE loss
    loss = criterion(y_predict, y_train_tensor)
    # Backward pass and gradient step
    optim.zero_grad()
    loss.backward()
    optim.step()
    if not epoch % 1000:
        # Print out the loss every 200 iterations
        print("epoch {}, loss {}".format(epoch, loss.item()))

In [16]:
from sklearn.metrics import r2_score

preds = model_1_h_l(X_test_tensor)

print(
    rf"The R^2 score of the 1 hidden layer model on the unseen dataset is {r2_score(preds.tolist(), y_test_tensor.tolist())}"
)

Two hidden layers

In [17]:
model_2_h_l = torch.nn.Sequential(
    torch.nn.Linear(X_train.shape[1], 128, bias=False),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 32, bias=False),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1, bias=False),
    torch.nn.ReLU(),
)

optim = torch.optim.Adam(model_2_h_l.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
num_epochs = 4000

for epoch in range(num_epochs + 1):
    # Pass training data through model
    y_predict = model_2_h_l(X_train_tensor)
    # Compute BCE loss
    loss = criterion(y_predict, y_train_tensor)
    # Backward pass and gradient step
    optim.zero_grad()
    loss.backward()
    optim.step()
    if not epoch % 1000:
        # Print out the loss every 200 iterations
        print("epoch {}, loss {}".format(epoch, loss.item()))

In [18]:
preds = model_2_h_l(X_test_tensor)

print(
    f"The R^2 score of the 2 hidden layer model on the unseen dataset is {r2_score(preds.tolist(), y_test_tensor.tolist())}"
)

From these experiments, it is evident that even a single hidden layer is capable of representing the data well. The value of the $r^2$ score suggests that the model is able to explain a significant portion of the variance in the dataset.
This suggests that evolving a neural network is a good choice for this problem, with a single hidden layer resulting in relatively small individuals.
Furthermore, the processing done above reduces the search space as the model is able to make predictions with smaller weights.

# Evolutionary algorithm

## Individuals and fitnesses


To avoid keeping an entire population of networks in memory, there will only exist a single neural network. 
The individuals will be represented by lists of floating-point numbers, with the length of the list matching the number of parameters in the network.

To evaluate an individual, its "genome" will be reshaped to fit the structure of the network. The individual's fitness will be the model's mean squared error on the dataset.

In [19]:
from collections import OrderedDict

HIDDEN_LAYER_SIZE = 256  # The number of hidden neurons used in the experiment above
MODEL = torch.nn.Sequential(
    OrderedDict(
        [
            ("fc1", torch.nn.Linear(X.shape[1], HIDDEN_LAYER_SIZE, bias=False)),
            ("relu1", torch.nn.ReLU()),
            ("fc2", torch.nn.Linear(HIDDEN_LAYER_SIZE, 1, bias=False)),
            ("relu2", torch.nn.ReLU()),
        ]
    )
)
IND_SIZE = len(torch.flatten(MODEL.fc1.weight)) + len(torch.flatten(MODEL.fc2.weight))
layer_1_size, layer_2_size = (
    len(torch.flatten(MODEL.fc1.weight)),
    len(torch.flatten(MODEL.fc2.weight)),
)
print(
    f"Each individual consists of {IND_SIZE} numbers representing the model weights, of which {layer_1_size} represent the 1st layer and {layer_2_size} the second"
)

Some simple tests of setting the model with an individual's weights

In [20]:
test_zeros = list(
    np.zeros(IND_SIZE)
)  # Based on experience from the practicals, it's better to keep individuals as lists as opposed to numpy arrays
test_ones = list(np.ones(IND_SIZE))
test_ones_and_zeros = list(
    np.concatenate((np.zeros(layer_1_size), np.ones(layer_2_size)))
)  # This should set all layer 1 weights to zeros and layer 2 weights to ones


def set_weights(individual):
    """
    Set the weights of the singleton model to the genes of the given individual
    :param individual: a list of size IND_SIZE
    """
    with torch.no_grad():
        MODEL.fc1.weight.data = torch.from_numpy(
            np.float32(individual[0:layer_1_size]).reshape(
                (HIDDEN_LAYER_SIZE, X.shape[1])
            )
        )
        MODEL.fc2.weight.data = torch.from_numpy(
            np.float32(individual[layer_1_size : layer_1_size + layer_2_size]).reshape(
                (1, HIDDEN_LAYER_SIZE)
            )
        )


set_weights(test_ones_and_zeros)
assert (
    torch.sum(MODEL.fc1.weight) == 0 and torch.sum(MODEL.fc2.weight) == layer_2_size
)  # Test the reshaping works correctly: first layer is all zeros and second layer is all ones

Evaluating the fitness of an individual - using the mean squared error as the metric

In [21]:
mse = torch.nn.MSELoss()


def evaluate(individual):
    with torch.no_grad():
        set_weights(individual)
        preds = MODEL(X_train_tensor)
    return (
        mse(preds, y_train_tensor).item(),
    )  # DEAP requires the use of tuples in the fitness


print(evaluate(test_ones), evaluate(test_zeros))

## The algorithm

The code below will populate a DEAP toolbox with the necessary functions to keep track and describe the success of a population of individuals.

To create a new individual, a list of size IND_SIZE will be populated with numbers sampled from a gaussian distribution of floating-point numbers. Keeping the mean of the distribution to a lower range allows the networks to be initialised with smaller weights and is a strategy commonly used in more conventional machine learning scenarios.

Given that the fitness of an individual is given by its mean squared error, the objective of the evolutionary algorithm is the minimisation of the individuals' fitnesses. For this, the weight of the fitness will be set to -1.

It should be noted that evolving a neural network is unlikely to benefit from crossover: changing the weights in such a drastic way can significantly affect the performance of the new individuals and lead to an unstable training curve. Instead, each individual in the population will mutate, allowing for more gradual changes in their genome.
It is also for this reason that a simple 1-dimensional list is sufficient to represent the individuals - a nested structure (e.g. a sublist per layer) will provide little benefit and will make the representation more complex.

Given the size of the individual and the network's sensitivity to changes in weights, the rate of mutation is likely to stay relatively low.

The best individuals will be entered in a hall of fame (a feature of DEAP), which will be used to retrieve the best ever recorded individual after the training is complete. This allows the algorithm to be more resilient to mutation negatively affecting the individuals' performance as the best genome will not be lost.

In [25]:
from deap import base, creator, tools

import random
from warnings import simplefilter, resetwarnings

simplefilter(
    "ignore", RuntimeWarning
)  # Hide warnings about overwriting classes in order to avoid showing the local username on the PDF
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
resetwarnings()  # Stop hiding runtime warnings

In [23]:
toolbox = base.Toolbox()
toolbox.register("attr_float", random.gauss, -1.0, 1.0)
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=IND_SIZE
)
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=25)

toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.5, indpb=0.003)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

hof = tools.HallOfFame(1)

## Investigating the effect of parameters on algorithm performance

Since crossover is not part of the algorithm, the main parameters influencing the performance of the model will be the population size, mutation rate and the number of participants in each tournament when selecting the offspring. To test their effects on the performance, it can be useful to setup a "grid search", where each experiment is ran several times to account for the stochastic nature of evolutionary algorithms. From there, the statistics of the gathered runs can be examined to determine the combination of parameters that is the most likely to lead to the best results when left to run for a large number of generations. To keep computational costs and run time of the search low, the number of generations in the experiment will be limited to 150.

Given that each individual is guaranteed to mutate as a whole, it is reasonable to keep the individual genes' mutation probabilities small - the relatively large size of the individuals combined with a high mutation rate will make evolution unstable. Therefore, mutation rates will not exceed 1% for an individual gene. (affecting around 46 values in each individual representing a 256-neuron wide network)

In [161]:
experiments = {}
NGEN_SEARCH = 150
for pop_size in [150, 200, 250]:
    for mutation_rate in [0.001, 0.002, 0.003, 0.01]:
        for tournament_size in [10, 15, 25]:
            experiment = f"Population size: {pop_size}, mutation rate: {mutation_rate}, tournament size: {tournament_size}"
            print(f"Conducting experiment {experiment}")
            for i in range(3):
                hof.clear()
                if experiment not in experiments.keys():
                    experiments[experiment] = []
                # Create population and evaluate initial fitnesses
                pop = toolbox.population(n=pop_size)
                hof = tools.HallOfFame(1)
                fitnesses = [toolbox.evaluate(indiv) for indiv in pop]
                for ind, fit in zip(pop, fitnesses):
                    ind.fitness.values = fit

                toolbox.register(
                    "mutate", tools.mutGaussian, mu=0.0, sigma=0.5, indpb=mutation_rate
                )
                toolbox.register(
                    "select", tools.selTournament, tournsize=tournament_size
                )

                for g in range(NGEN_SEARCH):
                    offspring = toolbox.select(pop, len(pop))
                    offspring = list(map(toolbox.clone, offspring))

                    for mutant in offspring:
                        toolbox.mutate(mutant)
                        del mutant.fitness.values

                    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
                    fitnesses = [toolbox.evaluate(indiv) for indiv in invalid_ind]
                    for ind, fit in zip(invalid_ind, fitnesses):
                        ind.fitness.values = fit

                    pop[:] = offspring
                    hof.update(pop)

                best_ind = hof.items[0]
                set_weights(best_ind)
                with torch.no_grad():
                    preds = MODEL(X_test_tensor)
                experiments[experiment].append(
                    {
                        "lowest fitness: ": best_ind.fitness.values[0],
                        "highest r^2 on test data": r2_score(
                            y_test_tensor.tolist(), preds.tolist()
                        ),
                    }
                )

Show the most successful experiments: sort the produced dict by the median highest $r^2$ score over the three runs

In [185]:
list(
    set(
        dict(
            sorted(
                experiments.items(),
                key=lambda item: np.median(
                    [it["highest r^2 on test data"] for it in item[1]]
                ),
                reverse=True,
            )
        ).keys()
    )
)[0:5]

It can be seen that a population of 200 is seen in three of the top 5 scoring experiments, making it a good choice for a potential longer run. Interestingly, the top result has been achieved with a relatively high mutation rate. While this could suggest that such high rates can be beneficial for a longer run, sticking with a lower rate (e.g. 0.003) can make the training process smoother and reduce the chances of a mutation making a good individual worse later in the training loop. Finally, a tournament size of 15 is present in 2/5 entries in the top 5, making it a good starting point for a longer run.

### Initialisation strategies

As mentioned above, initialisation is an important part of training a neural network. Therefore, it could be assumed that sampling from different distributions to create the initial population can affect the results.
While the search space for this parameter is quite large, it could be useful to look at 3 gaussian distributions with means of -1, 0 and 1 and a standard deviation of 1. This will provide a rough overview of the best sampling strategies. To change the distribution, the "attr_float" tool of the toolbox will be modified.

In [36]:
initialisation_experiments = {}
NGEN_SEARCH = 150
for mu in [-1.0, 0.0, 1.0]:
    toolbox.register("attr_float", random.gauss, mu=mu, sigma=1.0)
    toolbox.register(
        "individual",
        tools.initRepeat,
        creator.Individual,
        toolbox.attr_float,
        n=IND_SIZE,
    )
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    experiment = f"mu = {mu}"
    print("Running experiment ", experiment)
    for i in range(3):
        hof.clear()
        if experiment not in initialisation_experiments.keys():
            initialisation_experiments[experiment] = []
        # Create population and evaluate initial fitnesses
        pop = toolbox.population(n=200)
        hof = tools.HallOfFame(1)
        fitnesses = [toolbox.evaluate(indiv) for indiv in pop]
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        for g in range(NGEN_SEARCH):
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))

            for mutant in offspring:
                toolbox.mutate(mutant)
                del mutant.fitness.values

            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = [toolbox.evaluate(indiv) for indiv in invalid_ind]
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit

            pop[:] = offspring
            hof.update(pop)

        best_ind = hof.items[0]
        set_weights(best_ind)
        with torch.no_grad():
            preds = MODEL(X_test_tensor)
        initialisation_experiments[experiment].append(
            {
                "lowest fitness: ": best_ind.fitness.values[0],
                "highest r^2 on test data": r2_score(
                    y_test_tensor.tolist(), preds.tolist()
                ),
            }
        )

In [43]:
average_r2_scores = {}
for experiment, results in initialisation_experiments.items():
    print(
        f'The average best r^2 score for an initial gaussian distribution with {experiment} is {np.mean([result["highest r^2 on test data"] for result in results])}'
    )

From this, it is clear that an initial distribution with a mean of -1 allows the models to achieve better results on the test dataset. While the result for a mean of zero is not extreme and could suggest that improvement is possible after more generations, the results of an initial distribution with a mean of -1 suggest that the algorithm was able to begin producing viable individuals much quicker.

## Running the algorithm (with optimal parameters)

The following code block will put together the insights gathered during the experiments above and run the resulting algorithm for a longer number of generations. Then, the best individual will be tested and compared to a neural network trained via backpropagation to evaluate the model against a more commonly used algorithm.

In [215]:
NGEN = 1000
toolbox.register("attr_float", random.gauss, mu=-1.0, sigma=1.0)
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=IND_SIZE
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("select", tools.selTournament, tournsize=15)
toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.5, indpb=0.003)
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
display_logbook = (
    tools.Logbook()
)  # Logbook for displaying the scores as training happens - at a reduced rate
plotting_logbook = (
    tools.Logbook()
)  # Logbook for visualising curves - records every single generation
pop = toolbox.population(n=200)
hof.clear()
# Evaluate the fitness of the population prior to running the main algorithm
fitnesses = [toolbox.evaluate(indiv) for indiv in pop]
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit


display_logbook.header = "gen", "avg", "std", "min", "max"
plotting_logbook.header = "gen", "avg", "std", "min", "max"
for g in range(NGEN):
    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for mutant in offspring:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = [toolbox.evaluate(indiv) for indiv in invalid_ind]
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop[:] = offspring
    record = stats.compile(pop)
    hof.update(pop)
    plotting_logbook.record(gen=g, **record)
    if not g % 50:  # Display logbook every 50 epochs to avoid bloating the PDF
        display_logbook.record(gen=g, **record)
        print(display_logbook.stream)

In [216]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(plotting_logbook.select("min"))
ax.set_xlabel("Epoch", fontsize=20)
ax.set_ylabel("Fitness", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=18)
_ = ax.set_title("Best individual (lowest fitness) per epoch", fontsize=20)

It is clear that the performance of the indivuduals is consistently improving: the plot above shows the minimum fitness (i.e. the mean squared error on the training set) across the epochs.
The best individuals have been tracked by the `hof` variable, which allows accessing the best individual to have ever been recorded.

To evaluate the best individual on unseen data, the below cell will calculate the $r^2$ score of the individual on the portion of the original data held back for testing

In [218]:
best_ind = hof.items[0]
set_weights(best_ind)
print(f"The best individual' fitness is {best_ind.fitness.values[0]}")
with torch.no_grad():
    preds = MODEL(X_test_tensor)
    print(
        f"The individual's r^2 score on an unseen portion of the dataset is {r2_score(y_test_tensor.tolist(), preds.tolist())}"
    )

This score suggests that the evolved network is able to explain about 74% of the variance of the count of scooters.
While not a perfect fit to the data, this result is close to the score obtained via "conventional" training, suggesting that the evolutionary algorithm is capable of achieving similar quality results.
Another useful metric can be the mean absolute error on the testing dataset. This will allow us to see the performance of the network in the context of its target i.e. to see the mean number of scooters it is wrong by.

In [219]:
from sklearn.metrics import mean_absolute_error

preds_exponent = np.round(
    np.exp(preds)
)  # Having fractions of a scooter rented on a given day is impossible, convert to the nearest integer
actual_exponent = np.round(np.exp(y_test))
best_ind_mae = mean_absolute_error(actual_exponent, preds_exponent)
print(best_ind_mae)

On average, the trained network is mistaken by ~252 scooters. To see how this may affect the model in the "real world", it may be useful to plot the distribution of scooter counts.

In [220]:
preds_nn = model_1_h_l(X_test_tensor)
mae_nn = torch.nn.L1Loss()(torch.exp(preds_nn), torch.exp(y_test_tensor))

In [221]:
_, ax = plt.subplots(figsize=(16, 9))
ax.hist(np.exp(data_when_available["Count"]), bins=100)
ax.set_xlabel("Scooter count", fontsize=20)
ax.set_ylabel("Frequency", fontsize=20)
ax.plot(
    [best_ind_mae] * 400,
    range(400),
    label="Mean absolute error value (evolutionary algorithm)",
)
ax.plot(
    [mae_nn.item()] * 400,
    range(400),
    label="Mean absolute error value (conventional training)",
    color="purple",
)
ax.tick_params(axis="both", which="major", labelsize=18)
_ = ax.legend(fontsize=20)

The above plot shows that the model's mean error is greater than a large number of observations in the dataset. Despite that, the mean absolute error is close to that of the conventionally trained network.

While the results obtained in these experiments are not perfect, it is evident that the evolutionary approach is a powerful tool for evolving neural networks.

It can also be informative to look at the best individual's distribution of errors on the testing dataset as well as the median error

In [226]:
set_weights(best_ind)
with torch.no_grad():
    preds = MODEL(X_test_tensor)
preds_exponent = torch.round(torch.exp(preds))
actual_exponent = torch.round(torch.exp(y_test_tensor))
diffs = torch.abs(preds_exponent - actual_exponent)
diffs = diffs.squeeze().tolist()
median_diff = np.median(diffs)

_, ax = plt.subplots(figsize=(16, 12))
ax.hist(diffs, bins=100)
ax.plot(
    [median_diff] * 180,
    range(180),
    label=f"Median error ({median_diff})",
    color="red",
    linestyle="--",
)
ax.set_xlabel("Absolute error (scooters)", fontsize=20)
ax.set_ylabel("Frequency", fontsize=20)
ax.set_title("Distribution of absolute errors on the testing set", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=18)
_ = ax.legend(fontsize=20)

It is evident that the testing dataset contains several outliers which are driving up the mean error. This explains the difference between the mean and the median errors.

Finally, it can be useful to examine the performance of the conventionally trained 1-hidden-layer model compared to its evolved counterpart

In [227]:
preds_nn = model_1_h_l(X_test_tensor)
preds_nn_exp = torch.exp(preds_nn)
diffs_nn = torch.abs(preds_nn_exp - torch.exp(y_test_tensor))
diffs_nn = diffs_nn.squeeze().tolist()

In [228]:
_, ax = plt.subplots(figsize=(16, 9))
bp = ax.boxplot((diffs, diffs_nn), vert=False, widths=0.7)
for cap in bp["caps"]:
    cap.set(color="orange", linewidth=2, zorder=3)
ax.set_yticklabels(["Evolutionary\nalgorithm", "Conventional\ntraining"], fontsize=20)
ax.set_xlabel("Absolute error", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=18)
_ = ax.set_title("Distribution of errors for different training methods", fontsize=20)

The above plot, again, suggests that the results gathered using the evolutionary algorithm are of similar quality as the ones gathered using conventional training. 
The median errors across both methods are similar, though the 75th percentile error and interquartile range on the evolved network are greater. Both models show a relatively large number of outliers, suggesting that the fitting potential of the architecture itself is not enough to capture the full range of data, suggesting that a larger network could me more successful at fitting to the dataset.

Despite that, the small difference in median error suggest that evolving a neural network provided a relatively well-fitted model on par with a model trained via the usual backpropagation approach.

# Conclusion

Overall, it is clear that the evolutionary algorithm was able to produce a consistently improving population which, when fitted to a neural network, is able to closely match the results gained via common backpropagation algorithms. It should also be noted that the evolutionary algorithm took fewer epochs to achieve its result - requiring 900 generations compared to the 4000 epochs needed for backpropagation. While not a direct comparison due to each algorithm's different computational requirements, it is clear that evolutionary algorithms can quickly find a solution close to optimal given a suitably narrow search space provided by a compact representation.

The size of the network was kept relatively small, with the mean absolute error remaining an area of improvement should the model be used for any critical tasks. Despite that, it is commonly believed that larger (that is, wider, deeper or a combination of both) networks are able to fit the data better.
Care must be taken when training a larger network, especially that with a deeper architecture, as the size of each individual will grow at an exponential rate, leading to an increased memory and computational cost.

A future improvement to the algorithm could be seen in evolving the architecture of the network itself along with the weights, in an algorithm such as [NEAT](https://nn.cs.utexas.edu/?stanley:ec02). This, however, goes beyond the capabilities offered by DEAP.
The results presented here can therefore be seen as a "proof of concept" and a display of the capabilities of evolutionary algorithms.